In [2]:
import numpy as np
import torch

In [3]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [4]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)

In [5]:
def model(x):
    return x @ w.t() + b

In [7]:
pred = model(inputs)
def mse(t1,t2):
    diff = t1-t2
    return torch.sum(diff*diff)/diff.numel()
loss = mse(pred,targets)
print(loss)

tensor(63622.2891, grad_fn=<DivBackward0>)


In [9]:
loss.backward()
with torch.no_grad(): # We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()
#Before we proceed, we reset the gradients to zero by invoking the .zero_() method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke .backward on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results

In [13]:
# For multiple epochs
for i in range(120):
    pred = model(inputs)
    loss = mse(pred,targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()
pred = model(inputs)
pred

tensor([[ 58.0453,  71.4210],
        [ 77.2073,  94.9033],
        [128.6474, 144.2890],
        [ 26.0431,  42.0166],
        [ 90.2443, 106.3501]], grad_fn=<AddBackward0>)

In [22]:
# Linear regression using PyTorch built-ins
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as f

In [23]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [30]:
train_ds = TensorDataset(inputs, targets)
batch_size = 6
train_dl = DataLoader(train_ds,batch_size,shuffle=True)

# v
for xb, yb in train_dl: 
    print(xb)
    print(yb)

tensor([[ 73.,  66.,  44.],
        [ 87., 135.,  57.],
        [ 74.,  66.,  43.],
        [ 87., 134.,  58.],
        [ 68.,  96.,  71.],
        [101.,  44.,  37.]])
tensor([[ 57.,  69.],
        [118., 134.],
        [ 57.,  69.],
        [119., 133.],
        [104., 118.],
        [ 21.,  38.]])
tensor([[102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 68.,  97.,  70.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [ 92.,  87.,  64.]])
tensor([[ 22.,  37.],
        [ 81., 101.],
        [102., 120.],
        [ 80., 102.],
        [118., 132.],
        [ 82., 100.]])
tensor([[ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [103.,  43.,  36.]])
tensor([[103., 119.],
        [ 56.,  70.],
        [ 20.,  38.]])


In [25]:
model = nn.Linear(3,2) # 1st dimension is size of number of features, 2nd dimension is number of outputs
list(model.parameters())

[Parameter containing:
 tensor([[-0.2223, -0.0191,  0.3105],
         [-0.3711,  0.0647, -0.1696]], requires_grad=True),
 Parameter containing:
 tensor([-0.4776, -0.3466], requires_grad=True)]

In [26]:
pred = model(inputs)
pred

tensor([[ -4.6320, -30.3964],
        [ -2.5132, -39.2792],
        [ -4.3649, -33.8036],
        [-12.4840, -41.6927],
        [  4.0880, -31.6151],
        [ -4.8352, -30.8321],
        [ -2.1836, -39.5134],
        [ -4.2767, -34.3442],
        [-12.2808, -41.2569],
        [  4.6208, -31.4135],
        [ -4.3024, -30.6306],
        [ -2.7165, -39.7150],
        [ -4.6945, -33.5694],
        [-13.0168, -41.8942],
        [  4.2912, -31.1793]], grad_fn=<AddmmBackward0>)

In [27]:
loss_fn = f.mse_loss
loss = loss_fn(model(inputs), targets)
opt = torch.optim.SGD(model.parameters(), lr=1e-5) # stochastic gradient descent optimizer with defined learning rate

In [29]:
def model_fit(num_epochs, model, loss_fn, opt, train_dl):
    for epoch in range(num_epochs):
        for xb,yb in train_dl: # In each iteration, the data loader returns one batch of data with the given batch size
            pred = model(xb)
            loss = loss_fn(pred,yb)
            loss.backward()
            opt.step() # takes one step for optimizing
            opt.zero_grad() # reset gradients to zero

model_fit(120,model,loss_fn,opt, train_dl)

# Predict output
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.1737, 67.3737]], grad_fn=<AddmmBackward0>)